In [72]:
import json
import geojson
import pandas as pd


from OSMPythonTools.nominatim import Nominatim

from OSMPythonTools.overpass import Overpass
from OSMPythonTools.overpass import overpassQueryBuilder

import folium

from ipywidgets import HTML
from ipyleaflet import Map, WKTLayer, Marker, basemaps, basemap_to_tiles, Popup
import shapely.wkt

In [80]:
def wkt_to_geojson(wkt: str): 
    return geojson.Feature(geometry=shapely.wkt.loads(wkt))

In [128]:
nominatim = Nominatim()
area = nominatim.query('Auvergne rhone alpes', wkt=True)
area_center = shapely.wkt.loads(area.wkt()).centroid.coords[0][::-1]
area_geojson = wkt_to_geojson(area.wkt())
area.displayName()

'Auvergne-Rhône-Alpes, France métropolitaine, France'

In [141]:
selector = '"man_made"="bridge"'
query = overpassQueryBuilder(area, elementType=['node', 'way', 'relation'], selector=selector, includeGeometry=True, includeCenter=True)

overpass = Overpass()
result = overpass.query(query)

result_counter = dict(
    nodes=result.countNodes(), 
    ways=result.countWays(),
    areas=result.countAreas(),
    relations=result.countRelations(),
    elements=result.countElements()
    )

pd.Series(result_counter)

nodes           0
ways         1798
areas           0
relations       6
elements     1804
dtype: int64

In [143]:
markers = []
for element in result.elements():
    id = element.id()
    element_type = element.type()
    location = (element.lat(), element.lon())
    tags = json.dumps(element.tags())
    if element_type!='node':
        location = (element.centerLat(), element.centerLon())

    if not location[0]:
        print(location)
    
    # print(element_type)
    # marker = folium.map.Marker(location=location, tooltip=tags)
    # markers.append(marker)

(None, None)
(None, None)
(None, None)
(None, None)
(None, None)
(None, None)
(None, None)
(None, None)
(None, None)
(None, None)
(None, None)
(None, None)
(None, None)
(None, None)
(None, None)
(None, None)
(None, None)
(None, None)
(None, None)
(None, None)
(None, None)
(None, None)
(None, None)
(None, None)
(None, None)
(None, None)
(None, None)
(None, None)
(None, None)
(None, None)
(None, None)
(None, None)
(None, None)
(None, None)
(None, None)
(None, None)
(None, None)
(None, None)
(None, None)
(None, None)
(None, None)
(None, None)
(None, None)
(None, None)
(None, None)
(None, None)
(None, None)
(None, None)
(None, None)
(None, None)
(None, None)
(None, None)
(None, None)
(None, None)
(None, None)
(None, None)
(None, None)
(None, None)
(None, None)
(None, None)
(None, None)
(None, None)
(None, None)
(None, None)
(None, None)
(None, None)
(None, None)
(None, None)
(None, None)
(None, None)
(None, None)
(None, None)
(None, None)
(None, None)
(None, None)
(None, None)
(None, None)

In [101]:
map = folium.Map(location=area_center, tiles="openstreetmap", zoom_start=4)

layer_area = folium.features.GeoJson(data=area_geojson)
layer_area.add_to(map)

for marker in markers:
    marker.add_to(map)

layer_arcGIS = folium.TileLayer(
    tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
    attr = 'Esri',
    name = 'Esri Satellite',
    overlay = True,
    control = True
    )

layer_arcGIS.add_to(map)

layer_control = folium.LayerControl()
layer_control.add_to(map)

map.save('index.html')